In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install git+https://github.com/beroguedou/SpecAugment.git
#!pip install torchaudio
#!pip install torchsummary

In [18]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # To have better Traceback if CUDA fails

import os
import csv
import time
import logging
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
from decode import *
import torch
import torchaudio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed_value = 2020
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

In [4]:
UNITS = 128
VOCAB_SIZE = 30000
LIMIT = 1000
N_FRAMES = 1001
N_EPOCHS = 1
SPEC_DIM = (1, 39, N_FRAMES)
params = {'batch_size': None,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}

logging.basicConfig(filename='mytraining.log', format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',
                    level=logging.INFO)

# Initializing the datasets for training and validation
training_set = LibriSpeechDataset(limit=LIMIT, n_frames=N_FRAMES, version='train-clean-360')
dev_set = LibriSpeechDataset(limit=LIMIT, n_frames=N_FRAMES, version='dev-clean')

In [16]:
len(training_set)

1000

In [22]:
# Let's see a single utterance

LibriSpeechDataset(limit=LIMIT, n_frames=1000, version='train-clean-360')[0][1].shape

torch.Size([80])

In [7]:
# Defining an encoder, decoder optimizers and criterion
encoder = EncoderCONV2DRNN(device=device, batch_sz=params['batch_size'], hidden_size=UNITS, spec_dim=SPEC_DIM).to(device)
decoder = DecoderATTRNN(vocab_size=VOCAB_SIZE,  batch_sz=params['batch_size'], dec_units=UNITS, hidden_size=UNITS, 
                        encoder_timestamp=encoder.encoder_timestamp).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [8]:
len(dev_set)

1000

In [9]:
encoder.encoder_timestamp

332

In [10]:
# Train the model

global_trainer(N_EPOCHS, training_set, dev_set, params, encoder, decoder, encoder_optimizer,
               decoder_optimizer, criterion, device)

 ====== ====== ====== ====== ====== ======
      The model has 12604743 parameters
 ====== ====== ====== ====== ====== ======


Epoch        1: 100%|████████████████████| 41/41 [05:42<00:00,  8.34s/it, Train loss 5.3342 Eval loss 2.4947]

Time taken for the training 0.095369 hours



In [23]:

mfccs, references = training_set[1]
tokenizer =  BertTokenizer.from_pretrained('bert-base-uncased')
references = [tokenizer.convert_ids_to_tokens(ind) for ind in references.numpy().tolist()]
evaluate(mfccs.unsqueeze(0), references, 80, encoder, decoder, targ_lang_tokenizer=tokenizer, 
          device=device, beam_search=True, beam_width=10, enc_units=UNITS,
         encoder_timestamp=encoder.encoder_timestamp)

Input: ['[CLS]', '[CLS]', 'what', 'did', 'you', 'do', 'next', 'i', 'awakened', 'with', 'a', 'sudden', 'start', 'just', 'before', 'six', 'o', "'", 'clock', 'i', 'had', 'not', 'set', 'an', 'alarm', 'though', 'i', 'wanted', 'to', 'get', 'up', 'early', 'to', 'do', 'a', 'little', 'repair', 'job', 'i', 'had', 'promised', 'for', 'early', 'this', 'morning', '[SEP]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


Predicted translation: ['[CLS]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [ ]:
# Vérifications

# 1 - Data Augmentation
# 2 - Encoder
# 3 - Attention Mechanism Bahdanau Audio
# 4 - Smoothing and Topk to the attention
# 5 - Decoder 
# 6 - Métrique BLEU


In [ ]:
#!pip freeze > recquirements.txt